In [1]:
from collections import Iterable
import numpy as np
import imutils
import pickle
import time 
import cv2
import csv

C:\Users\HP.DESKTOP-5IHNLMQ\AppData\Local\Temp\ipykernel_13256\1740433666.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [2]:
def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
                
        else:
            yield item

In [3]:
embeddingFile = "output/embeddings.pickle"
embeddingModel = "openface_nn4.small2.v1.t7"
recognizerFile = "output/recognizer.pickle"
labelEncFile = "output/le.pickle"
conf = 0.5

In [4]:
print("[INFO] loading face detector....")
prototxt = "model/deploy.prototxt"
model = "model/res10_300x300_ssd_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(prototxt, model)


[INFO] loading face detector....


In [ ]:
print("[INFO] loading face recognizer....")
embedder = cv2.dnn.readNetFromTorch(embeddingModel)

recognizer = pickle.loads(open(recognizerFile, "rb").read())
le = pickle.loads(open(labelEncFile, 'rb').read())

Roll_Number = ""
box = []
print("Starting video stream...")
cam = cv2.VideoCapture(0)
time.sleep(2.0)

while True:
    _, frame = cam.read()
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    detector.setInput(imageBlob)
    detections = detector.forward()

    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence > conf:

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            if fW < 20 or fH < 20:
                continue

            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]
            with open('student.csv', 'r') as csvFile:
                reader = csv.reader(csvFile)
                for row in reader:
                    box = np.append(box, row)
                    name = str(name)
                    if name in row:
                        person = str(row)
                        print(name)
                listString = str(box)
                print(box)
                if name in listString:
                    singleList = list(flatten(box))
                    listlen = len(singleList)
                    Index = singleList.index(name)
                    name = singleList[Index]
                    Roll_Number = singleList[Index + 1]
                    print(Roll_Number)

            text = "{} : {} : {:.2f}%".format(name, Roll_Number, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                          (0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

[INFO] loading face recognizer....
Starting video stream...
Dinesh
['222.5328505039215' '196.08302414417267' '356.82263374328613'
 '374.57115948200226' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['223.13196659088135' '195.06065994501114' '354.8389792442322'
 '360.2423965930939' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['229.59516048431396' '193.24296712875366' '360.94722747802734'
 '355.1382601261139' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['230.68664073944092' '193.46795082092285' '362.3211622238159'
 '358.49552750587463' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['233.87914896011353' '192.68651604652405' '372.71400690078735'
 '388.94476890563965' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['235.4712724685669' '193.81855577230453' '376.9996404647827'
 '387.30926513671875' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['24

Amol
['209.86579656600952' '198.5426366329193' '342.9041862487793'
 '368.88023614883423' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
665544
Dipak
['203.23251485824585' '203.7794217467308' '332.43666887283325'
 '368.69258880615234' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
225588
Amol
['195.72640657424927' '201.33004188537598' '329.1723847389221'
 '372.9159414768219' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
665544
Dinesh
['198.83229732513428' '202.17904150485992' '327.4494409561157'
 '370.0128525495529' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Amol
['193.21264028549194' '201.6476958990097' '323.93020391464233'
 '369.7261780500412' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
665544
Amol
['186.53433322906494' '199.18068051338196' '315.84423780441284'
 '365.9314155578613' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
665544
Amol
['184.78193879127502' '200.0201851129532' '314.45252895355225'
 '364.8004621267319' 'Din

Amol
['198.6471712589264' '171.11539989709854' '332.59384632110596'
 '360.0220263004303' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
665544
Dinesh
['200.52840113639832' '171.64796590805054' '331.57339096069336'
 '358.91448855400085' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['199.70346093177795' '170.7219198346138' '332.7701210975647'
 '360.5088472366333' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['200.05540251731873' '172.96254336833954' '332.91141986846924'
 '361.681267619133' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Amol
['199.70240592956543' '173.7365484237671' '332.1364760398865'
 '359.5004975795746' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
665544
Amol
['198.36650490760803' '174.5275318622589' '333.60217809677124'
 '362.3609662055969' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
665544
Amol
['198.36650490760803' '174.5275318622589' '333.60217809677124'
 '362.3609662055969'

Dinesh
['208.2620143890381' '187.44054436683655' '332.7126145362854'
 '347.2955882549286' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['208.42933058738708' '187.14780807495117' '332.6873302459717'
 '348.00851941108704' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['208.2447588443756' '187.46869415044785' '333.1639766693115'
 '345.0332522392273' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['207.66913890838623' '184.73459780216217' '333.33635330200195'
 '348.7366050481796' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['207.01417922973633' '186.76170408725739' '333.757746219635'
 '347.2958832979202' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['207.2812557220459' '186.00545525550842' '334.2955470085144'
 '348.09316992759705' 'Dinesh' '11223300' 'Dipak' '225588' 'Amol' '665544']
11223300
Dinesh
['208.32712054252625' '185.73157489299774' '333.05883407592773'
 '346.